## Analyse the manually labelled results from SOCMapper

In [2]:
import pandas as pd

import random

In [3]:
random_job_matches = pd.read_csv("s3://prinz-green-jobs/outputs/data/ojo_application/evaluation/soc_evaluation_random_sample_manual_121023.csv")
print(len(random_job_matches))

full_common_job_matches = pd.read_csv("s3://prinz-green-jobs/outputs/data/ojo_application/evaluation/soc_evaluation_sample_manual_121023.csv")
common_job_matches = full_common_job_matches[full_common_job_matches['labelled']==False] # Bad column name on my part! This is because we didn't label all of them in this csv
print(len(common_job_matches))

200
300


In [4]:
random_job_matches.rename(columns={
    'Quality of soc_2020_6_name (0-cant say, 1-poor, 2-ok, 3-excellent)': "soc_6_quality",
    'If match_type = "Multiple 4-digit 2020 socs" - Quality of soc_2020_4_name (0-cant say, 1-poor, 2-ok, 3-excellent)': "soc_4_quality"
}, inplace=True)


In [5]:
common_job_matches.rename(columns={
    'Quality of soc_2020_6_name (0-cant say, 1-poor, 2-ok, 3-excellent)': "soc_6_quality",
    'If match_type = "Multiple 4-digit 2020 socs" - Quality of soc_2020_4_name (0-cant say, 1-poor, 2-ok, 3-excellent)': "soc_4_quality"
}, inplace=True)

/Users/elizabethgallagher/miniconda3/envs/dap_prinz_green_jobs/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


## The random sample

In [6]:
random_job_matches['match_type'].value_counts(normalize=True)

Top 6-digit 2020 soc          0.595
None found                    0.355
Multiple 4-digit 2020 socs    0.050
Name: match_type, dtype: float64

In [7]:
filt_d = random_job_matches[
          ((random_job_matches['match_type']=='Top 6-digit 2020 soc') & (random_job_matches['soc_6_quality']!=0))
          ]
print(len(filt_d))
filt_d['soc_6_quality'].value_counts(normalize=True)

118


3.0    0.658120
2.0    0.230769
1.0    0.111111
Name: soc_6_quality, dtype: float64

In [8]:
print(len(random_job_matches[random_job_matches['match_type']=='Multiple 4-digit 2020 socs']))
random_job_matches[random_job_matches['match_type']=='Multiple 4-digit 2020 socs']['soc_4_quality'].value_counts(normalize=True)

10


3.0    0.8
2.0    0.1
1.0    0.1
Name: soc_4_quality, dtype: float64

In [9]:
random_job_matches['match_type'].value_counts()

Top 6-digit 2020 soc          119
None found                     71
Multiple 4-digit 2020 socs     10
Name: match_type, dtype: int64

## The top job title sample

In [10]:
full_common_job_matches["match_type"].value_counts(normalize=True)

Top 6-digit 2020 soc          0.892
None found                    0.073
Multiple 4-digit 2020 socs    0.035
Name: match_type, dtype: float64

In [11]:
filt_d = common_job_matches[
          ((common_job_matches['match_type']=='Top 6-digit 2020 soc') & (common_job_matches['soc_6_quality']!=0))]
print(len(filt_d))
filt_d['soc_6_quality'].value_counts(normalize=True)

255


3.0    0.819608
2.0    0.101961
1.0    0.078431
Name: soc_6_quality, dtype: float64

In [12]:
filt_d = common_job_matches[common_job_matches['match_type']=='Multiple 4-digit 2020 socs']
print(len(filt_d))
filt_d['soc_4_quality'].value_counts(normalize=True)

20


3.0    0.95
2.0    0.05
Name: soc_4_quality, dtype: float64

## Combine and get the bad ones

In [14]:
job_matches = common_job_matches.append(random_job_matches)
len(job_matches)

500

In [15]:
poor_10 = job_matches[job_matches['soc_6_quality']==1][['ojo_job_title', 'num_job_ads', 'prop_job_ads', 'soc_2020_6_name', 'occ_matched', 'match_prob']].sample(10, random_state=42)
poor_10

,ojo_job_title,num_job_ads,prop_job_ads,soc_2020_6_name,occ_matched,match_prob
189,Programme Administrators,1.0,1.390000e-07,Functional managers and directors n.e.c.,programme manager,0.783059
198,Academic Mentor,2847.0,3.959420e-04,Learning and behaviour mentors,learning mentor,0.845167
103,Electronics Assembly Technician - Oxford - &#1...,2.0,2.780000e-07,Metal working production and maintenance fitte...,assembly engineer,0.671789
258,Senior Administrator,2315.0,3.219550e-04,Registrars,senior registration administrator,0.843326
148,Census officer,3547.0,4.932930e-04,Office managers,census district manager,0.796723
153,Sales Negotiator,3442.0,4.786910e-04,Estate agents,sales negotiator,1.000000
275,Operative,2201.0,3.061010e-04,Textile process operatives n.e.c.,general operative,0.765703
12,Business Case Manager Business,1.0,1.390000e-07,National government administrative occupations...,case manager,0.763794
162,Census officer - Customer service,3304.0,4.594980e-04,Office managers,census district manager,0.741204
17,Production Operative,16113.0,2.240890e-03,Printing machine assistants,finishing operative,0.761396


In [16]:
excellent_10 = job_matches[job_matches['soc_6_quality']==3][['ojo_job_title', 'num_job_ads', 'prop_job_ads', 'soc_2020_6_name', 'occ_matched', 'match_prob']].sample(10, random_state=42)
excellent_10

,ojo_job_title,num_job_ads,prop_job_ads,soc_2020_6_name,occ_matched,match_prob
11,Care Assistant - Bank - Care Home,22444.0,3.121363e-03,Domiciliary care workers,home care assistant,0.781899
154,Pastry Demi Chef de Partie,1.0,1.390000e-07,Chefs,chef de partie,0.787605
192,Forklift Driver,2922.0,4.063720e-04,Fork-lift truck drivers,fork lift truck driver,0.881016
10,Finance ManagerRB,1.0,1.390000e-07,Company secretaries and finance managers,finance manager,0.847019
45,Service Engineer Carpentry and Decorating,1.0,1.390000e-07,Painters and decorators,decorating contractor,0.723216
208,Senior Software Engineer,2681.0,3.728560e-04,Software developers,senior software engineer,1.000000
188,Change Business Analyst - FMCG experience,1.0,1.390000e-07,Business analysts,business change analyst,0.690415
96,Private Client Solicitor,5338.0,7.423740e-04,Solicitors and lawyers n.e.c.,solicitor,0.750278
265,Internal Sales Executive,2281.0,3.172260e-04,Business sales executives,sales executive,0.846217
40,HR Advisor,10386.0,1.444416e-03,Human resources advisors,human resources adviser,0.853155


In [17]:
good_10 = job_matches[job_matches['soc_6_quality']==2][['ojo_job_title', 'num_job_ads', 'prop_job_ads', 'soc_2020_6_name', 'occ_matched', 'match_prob']].sample(10, random_state=42)
good_10

,ojo_job_title,num_job_ads,prop_job_ads,soc_2020_6_name,occ_matched,match_prob
189,Domestic Assistant,2934.0,4.080410e-04,Commercial cleaners,domestic assistant,1.000000
98,Holiday Club Admin Manager,1.0,1.390000e-07,Hotel and accommodation managers and proprietors,holiday centre manager,0.786236
150,Training and Support Manager,1.0,1.390000e-07,Education managers,learning support manager,0.804226
112,Digital Marketing Executive,4554.0,6.333400e-04,Marketing consultants,digital marketing executive,1.000000
106,Operations Manager -Commercial Insurance,1.0,1.390000e-07,Financial managers and directors n.e.c.,insurance company manager,0.789530
67,Field Service Engineer,7272.0,1.011342e-03,Telecoms and related network installers and re...,home service field engineer,0.873159
159,Tutor,3370.0,4.686770e-04,Higher education teaching professionals n.e.c.,course tutor,0.917003
192,Assistant Manager - Truro,2.0,2.780000e-07,Other administrative occupations n.e.c.,manager's assistant,0.775355
52,Marketing Executive,8363.0,1.163071e-03,Marketing consultants,marketing executive,1.000000
52,Chartered Financial Advisor - Berkshire,2.0,2.780000e-07,Financial accountants,chartered management accountant,0.698416


## Which job titles don't match

In [18]:
not_matched_job_titles = list(set(full_common_job_matches[full_common_job_matches["match_type"]=='None found']["ojo_job_title"].tolist() +random_job_matches[random_job_matches["match_type"]=='None found']["ojo_job_title"].tolist()))

In [19]:
random.seed(42)
print(random.sample(not_matched_job_titles, 20))

['Driver - London', 'Paid Emails - Work From Home', 'Agile Transformation Consultant', '202207 - Detective Constable Pathway', 'C++ Software Engineer QT', 'Supply and PPA Teachers needed in New Cross, Lewisham', '7.5T Driver', 'Class 1 Driver', 'KS1 Teacher', 'Part Home Based Block Manager - Chichester', 'Senior Community Host - Woking', '202203 PC-DHEP Police Constable Degree Holder Entry Programme', 'CELTA Certified English Teacher remote - Doncaster', 'Wellbeing Co-ordinator', 'Digital Designer- Outside IR35- Contract', 'Graduate Programme', 'Full-Stack Engineer React, Node', 'Team Executive - Funds', 'Private Patient Sales Executive', 'Blinds and Curtains Installer']
